In [1]:
# load weights


# write function to make predictions from weights and return intermediate values


# double check that my predictions are the same as saved models

In [9]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.ticker import ScalarFormatter
import os
import pandas as pd
import time
from datetime import datetime
import sys
import seaborn as sns
import scipy.io
import subprocess
import winsound
import pickle
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
print("TensorFlow successfully installed.")
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
print(sys.version, "\n")
now = datetime.now()
print("last run on " + str(now))

# define directories
baseDir = os.getcwd()
dataDir = r'D:\MothSimulations\11c-AggressiveManeuver\Qstore\hws_am_con'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
randomRawData = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\PythonGeneratedData\TrainingData'
if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)


TensorFlow successfully installed.
The installed version of TensorFlow includes GPU support.
3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)] 

last run on 2019-05-24 13:27:43.953595


In [10]:
# import
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras import regularizers

# Keras callcacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras.backend as K

In [11]:
X_train = np.random.rand(100,10 )
Y_train = np.random.rand(X_train.shape[0], 7) + np.sum(X_train)

In [12]:
K.clear_session()
inputs = Input(shape=(X_train.shape[1],))    

# add layers
x = Dense(500, activation='tanh', )(inputs)
x = Dense(566, activation='tanh')(x)
x = Dense(567, activation='tanh')(x)
predictions = Dense(Y_train.shape[1], activation='linear')(x)

# create model
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss="mean_squared_error", optimizer = "rmsprop", metrics = ['mse'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               5500      
_________________________________________________________________
dense_2 (Dense)              (None, 566)               283566    
_________________________________________________________________
dense_3 (Dense)              (None, 567)               321489    
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 3976      
Total params: 614,531
Trainable params: 614,531
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(X_train, Y_train, epochs = 3, verbose = 2, 
                        batch_size=16)

Epoch 1/3
 - 0s - loss: 241342.6794 - mean_squared_error: 241342.6794
Epoch 2/3
 - 0s - loss: 230555.5988 - mean_squared_error: 230555.5988
Epoch 3/3
 - 0s - loss: 225781.2975 - mean_squared_error: 225781.2975


In [15]:
wts = model.get_weights().copy()
# wts

# for ii in range(len(wts)):
#     print(wts[ii], "\n")

In [16]:
model.predict(np.array([[1,2,3,4,5,6,7,8,9,10]]))

array([[27.963383, 28.111612, 26.493656, 29.12884 , 28.487183, 27.65596 ,
        27.848248]], dtype=float32)

In [17]:
def returnLayerValues(weights, inputData):
    '''
    Returns intermediate layer values from fitted network
    
    The fitted network uses "tanh" activation for hidden layer and
        "linear" activation for the final layer
        
    Params:
        weights (list): list of weights from a fitted model
        inputData (array or Data frame): input data that will be run through the network
    
    Returns:
        layer values (list): values of all of the units in the network.
            - the 0'th item in the list is the input data
            - the final item in the list is the final prediction
    '''
    LayerValues = [inputData.astype("float64")]
    for layerNum in np.arange(0, len(wts), 2):
        
        # calculate dot product and add bias
        nextLayer = np.dot(LayerValues[-1], wts[layerNum]) + wts[layerNum+1]
        
        if layerNum != (len(wts)-2):
            # apply activation function, except for final layer
            nextLayer = np.tanh(nextLayer)
        
        # append to list
        LayerValues.append(nextLayer)
        
    return(LayerValues)

In [27]:
# this function returns values of all layers
def returnLayerValues2(weights, inputData):
    '''
    Returns all layer values from fitted network (including input and output)
    
    The fitted network uses "tanh" activation for hidden layers and
        "linear" activation for the final layer
        
    Params:
        weights (list): list of weights from a fitted model
        inputData (array or Data frame): input data that will be run through the network
    
    Returns:
        layer values (list): values of all of the units in the network.
            - the 0'th item in the list is the input data
            - the final item in the list is the final prediction
    '''
    
    LayerValues = [inputData.astype("float64")]
    for layerNum in np.arange(0, len(wts), 2):
        
        # this version combines weights and biases into single matrix
        nextInput =  np.hstack([np.array([[1]]), LayerValues[-1]])
        wts_and_bias = np.vstack([wts[layerNum+1], wts[layerNum]])
        
        # calculate dot product and add bias
        nextLayer = np.dot(nextInput, wts_and_bias)
        
        if layerNum != (len(wts)-2):
            # apply activation function, except for final layer
            nextLayer = np.tanh(nextLayer)
        
        # append to list
        LayerValues.append(nextLayer)
        
    return(LayerValues)

In [29]:
wts = model.get_weights().copy()
intermediateValues = returnLayerValues(wts, inputData= np.array([[1,2,3,4,5,6,7,8,9,10]]))
intermediateValues

[array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]),
 array([[-0.95060035,  0.73090569,  0.92555474,  0.93525605,  0.73308881,
          0.88345868, -0.72785576, -0.9984618 , -0.7621206 , -0.9573174 ,
          0.93519419,  0.99959038,  0.99407362, -0.8817773 ,  0.82130208,
         -0.48420636,  0.85414298, -0.56576346, -0.74147101,  0.881499  ,
         -0.87751135,  0.92177726, -0.64456637,  0.5513546 ,  0.95861065,
          0.9627685 ,  0.99339974, -0.81757951,  0.71999798, -0.98874333,
         -0.87184814,  0.89552934, -0.97357109,  0.99380715, -0.85009765,
         -0.97386688, -0.99601516, -0.91492469,  0.92087869, -0.98677162,
          0.99264342,  0.98230568, -0.88494556,  0.80731088,  0.93940721,
          0.93022448, -0.3776354 , -0.95066184,  0.77999077,  0.69470749,
         -0.89935773,  0.96824024,  0.27950445,  0.65816965,  0.11427486,
          0.85634733,  0.72963965,  0.74392718,  0.98695517, -0.99427107,
         -0.15255588,  0.35898409, -0.97414177,  0

In [30]:
intermediateValues2 = returnLayerValues2(wts, inputData= np.array([[1,2,3,4,5,6,7,8,9,10]]))

In [33]:
np.allclose(intermediateValues[-1], intermediateValues2[-1])

True

In [34]:
intermediateValues[-1]-intermediateValues2[-1]

array([[-7.10542736e-15,  1.06581410e-14,  1.06581410e-14,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [ ]:
# check with actual data


In [332]:
# read in data
trainDF = pd.read_hdf(os.path.join(dataOutput, "concatenatedRandomICs.h5"), key = "data")

In [333]:
# rename columns to be consistent with other code
trainDF.rename(columns={"x0" : "x_0", "y0" : "y_0", "phi0" : "phi_0", "theta0" : "theta_0", 
                        "xf" : "x_99", "yf" : "y_99", "phif" : "phi_99", "thetaf" : "theta_99", 
                        "xd0" : "x_dot_0", "yd0" : "y_dot_0", "phid0" : "phi_dot_0", "thetad0": "theta_dot_0", 
                        "xdf" : "x_dot_99", "ydf": "y_dot_99", "phidf": "phi_dot_99", "thetadf": "theta_dot_99", 
                        "tau0" : "tau"}, inplace=True)

In [28]:
inputData= np.array([[1,2,3,4,5,6,7,8,9,10]])
inputData_bias = np.hstack([np.array([[1]]), np.array([[1,2,3,4,5,6,7,8,9,10]])])

wts_bias = np.vstack([wts[1], wts[0]])


nextLayer = np.dot(inputData_bias, wts_bias)


In [334]:
# convert to fx and fy
trainDF["Fx"] = trainDF.F * np.cos(trainDF.alpha)
trainDF["Fy"] = trainDF.F * np.sin(trainDF.alpha)

In [335]:
# make dataset
X = trainDF.loc[:, [ "phi_0", "theta_0", 
                    "x_99", "y_99", "phi_99", "theta_99", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0"]]

Y = trainDF.loc[:, ["Fx", "Fy", "tau", "x_dot_99", "y_dot_99", 
                    "phi_dot_99", "theta_dot_99"] ]

In [350]:
# test train split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state = 123)

In [351]:
# scale data 
scalerX = MinMaxScaler([-0.5, 0.5])  
scalerY = MinMaxScaler([-0.5, 0.5])  

# Don't cheat - fit only on training data
scalerX.fit(Xtrain)  
scalerY.fit(Ytrain) 

Xtrain_scaled = scalerX.transform(Xtrain)  
Ytrain_scaled = scalerY.transform(Ytrain)  

# apply same transformation to test data
Xtest_scaled = scalerX.transform(Xtest)
Ytest_scaled = scalerY.transform(Ytest)

In [352]:
NNetVisDataPath = r"D:\Dropbox\AcademiaDropbox\UW\NNetVisualization\ExampleData"

In [356]:
np.random.seed(1239039)
sampleInts = np.random.randint(0, Xtrain_scaled.shape[0], size = 1000)

In [357]:
smallX_train = Xtrain_scaled[sampleInts, :]
smallY_train = Ytrain_scaled[sampleInts, :]

In [363]:
# save to csv
xtsm = pd.DataFrame(smallX_train, columns = X.columns)
ytsm = pd.DataFrame(smallY_train, columns = Y.columns)


xtsm.to_csv(os.path.join(NNetVisDataPath, "X_train_small.csv"), index = False)
ytsm.to_csv(os.path.join(NNetVisDataPath, "Y_train_small.csv"), index = False)

In [365]:
# read in data and model
dpth = r"D:\Dropbox\AcademiaDropbox\UW\NNetVisualization\ExampleData\Opt_rmsprop__Dro_0.0__Num_20_16__Wei_0.h5"

from keras.models import load_model
model = load_model(dpth)

In [368]:
wts = model.get_weights().copy()
model.predict(xtsm)

array([[ 0.07868618,  0.05531172, -0.4204175 , ..., -0.2919854 ,
         0.01584185,  0.00471532],
       [ 0.20772386,  0.17999463, -0.19987795, ...,  0.06676452,
        -0.19427305, -0.20139787],
       [-0.01532531, -0.5210961 ,  0.44184887, ...,  0.02331303,
         0.41064593,  0.40982944],
       ...,
       [-0.01556098, -0.38132113,  0.35222876, ..., -0.14111082,
         0.11535618,  0.11606424],
       [-0.30363825, -0.3739403 ,  0.40084484, ...,  0.32999206,
         0.08843673,  0.08833369],
       [-0.03489161, -0.37062043, -0.4276588 , ..., -0.05476546,
        -0.01598368, -0.02264309]], dtype=float32)

In [369]:
intermediateValues = returnLayerValues(wts, inputData= xtsm)

In [373]:
intermediateValues[-1]

array([[ 0.07868646,  0.05531151, -0.42041834, ..., -0.29198529,
         0.01584188,  0.00471533],
       [ 0.20772348,  0.1799945 , -0.19987852, ...,  0.06676457,
        -0.19427305, -0.20139787],
       [-0.01532537, -0.52109632,  0.44184895, ...,  0.02331301,
         0.41064588,  0.40982945],
       ...,
       [-0.01556104, -0.38132154,  0.35222881, ..., -0.14111085,
         0.11535617,  0.11606422],
       [-0.30363825, -0.37394039,  0.40084487, ...,  0.32999208,
         0.08843675,  0.08833368],
       [-0.03489144, -0.37062073, -0.42765931, ..., -0.05476542,
        -0.01598369, -0.0226431 ]])

In [374]:
diffs = model.predict(xtsm) - intermediateValues[-1]

In [377]:
np.max(abs(diffs))

2.2758603189787507e-06

In [379]:
diffs = model.predict(xtsm) - ytsm
np.max(abs(diffs))

Fx              0.206681
Fy              0.249421
tau             0.129171
x_dot_99        0.184459
y_dot_99        0.188683
phi_dot_99      0.024570
theta_dot_99    0.022247
dtype: float64

In [323]:
# return weights from Keras
from keras.models import Model

inputData = np.array([[1,2,3,4,5,6,7,8,9,10]])

layer_name = 'dense_4'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(inputData)
intermediate_output

array([[1.7913353, 1.6383936, 2.6490834, 1.9456788, 1.945498 , 1.6709379,
        1.0960532]], dtype=float32)

In [270]:
model.predict(np.array([[4.332323, 5.23232]], dtype = "float64"))

array([[-0.52946097]], dtype=float32)

In [205]:
inputData = np.array([[4,5]])
# merge bias and weights matrices
LayerValues = []
    
# add bias to input data
inputData = np.hstack([np.array([1], dtype = "float32").reshape(-1,1), inputData] )

LayerValues.append(inputData)

jj = 0

wtsConcatenated = [np.vstack([wts[ii + 1], wts[ii]]) for ii in np.arange(0, len(wts), 2)]
nextLayer = np.dot(LayerValues[jj], wtsConcatenated[jj]).astype("float32")
nextLayer

array([[9.891574, 9.837484]], dtype=float32)

In [51]:
ii = 2
#wts[0] = np.vstack((np.ones(2, dtype = "float32"), wts[0]))

np.vstack([wts[ii + 1], wts[ii]])

array([[0.00736949],
       [1.3782268 ],
       [1.347521  ]], dtype=float32)

In [53]:
wtsConcatenated = [np.vstack([wts[ii + 1], wts[ii]]) for ii in np.arange(0, len(wts), 2)]

In [65]:
returnLayerValues(wts, inputData= np.array([[1,3]]))

[array([[1, 1, 3]])]